# 5. 금융 데이터 수집하기 (기본)

이번 장에서는 금융 데이터를 구하기 위해 필요한 주식티커와 섹터별 구성종목을 크롤링하는 방법에 대해 알아보겠습니다.

## 5.1 한국거래소의 업종분류 현황 및 개별지표 크롤링

앞 장의 예제를 통해 네이버 금융에서 주식티커를 크롤링하는 방법을 살펴보았습니다. 그러나 이 방법은 지나치게 복잡하고 시간이 오래 걸립니다. 반면 한국거래소에서 제공하는 업종분류 현황과 개별종목 지표 데이터를 이용하면 훨씬 간단하게 주식티커 데이터를 수집할 수 있습니다.

- KRX 정보데이터시스템 http://data.krx.co.kr/ 에서 [기본통계 → 주식 → 세부안내] 부분
- [12025] 업종분류 현황: http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506
- [12021] 개별종목: http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020502

해당 데이터들을 크롤링이 아닌 [Excel] 버튼을 클릭해 엑셀 파일로 받을 수도 있습니다. 그러나 매번 엑셀 파일을 다운로드하고 이를 불러오는 작업은 상당히 비효율적이며, 크롤링을 이용한다면 해당 데이터를 직접 불러올 수 있습니다.

### 5.1.1 업종분류 현황 크롤링

먼저 업종분류 현황에 해당하는 페이지에 접속한 후 개발자 도구 화면을 열고 [다운로드] 버튼을 클릭한 후 [CSV]를 누릅니다. [Network] 탭에는 generate.cmd와 download.cmd 두 가지 항목이 있습니다. 거래소에서 엑셀 데이터를 받는 과정은 다음과 같습니다.

1. http://data.krx.co.kr/comm/fileDn/download_excel/download.cmd 에 원하는 항목을 쿼리로 발송하면 해당 쿼리에 해당하는 OTP(generate.cmd)를 받게 됩니다.
2. 부여받은 OTP를 http://data.krx.co.kr/에 제출하면 이에 해당하는 데이터(download.cmd)를 다운로드하게 됩니다.

먼저 1번 단계를 살펴보겠습니다.

```{figure} image/05_crawl_practice_krx_sector.png
---
name: 05_crawl_practice_krx_sector
---
OTP 생성 부분
```

General 항목의 Request URL의 앞부분이 원하는 항목을 제출할 주소입니다. Form Data에는 우리가 원하는 항목들이 적혀 있습니다. 이를 통해 POST 방식으로 데이터를 요청함을 알 수 있습니다.

다음으로 2번 단계를 살펴보겠습니다.

```{figure} image/05_crawl_practice_krx_sector2.png
---
name: 05_crawl_practice_krx_sector2
---
OTP 제출 부분
```

General 항목의 Request URL은 OTP를 제출할 주소입니다. Form Data의 OTP는 1번 단계에서 부여받은 OTP에 해당합니다. 이 역시 POST 방식으로 데이터를 요청합니다.

위 과정을 코드로 나타내면 다음과 같습니다.

In [1]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
  'mktId': 'STK',
  'trdDd': '20210108',
  'money': '1',
  'csvxls_isNo': 'false',
  'name': 'fileDown',
  'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

print(otp)

i638upk45ZUhZAhVnc7yfoyoktkFbZQwhZTrLxeITMwRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eaZ8NA5R6MzhK2oIoU32nFwtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


1. gen_otp_url에 원하는 항목을 제출할 URL을 입력합니다.
2. 개발자 도구 화면에 나타는 쿼리 내용들을 딕셔너리 형태로 입력합니다. 이 중 mktId의 STK는 코스피에 해당하는 내용이며, 코스닥 데이터를 받고자 할 경우 KSQ를 입력해야 합니다.
3. 헤더 부분에 리퍼러(Referer)를 추가합니다. 리퍼러란 링크를 통해서 각각의 웹사이트로 방문할 때 남는 흔적입니다. 거래소 데이터를 다운로드하는 과정을 살펴보면 첫 번째 URL에서 OTP를 부여받고, 이를 다시 두번째 URL에 제출했습니다. 그런데 이러한 과정의 흔적이 없이 OTP를 바로 두번째 URL에 제출하면 서버는 이를 로봇으로 인식해 데이터를 반환하지 않습니다. 따라서 헤더 부분에 우리가 거쳐온 과정을 흔적으로 남겨야 데이터를 반환하게 됩니다.
4. `post()` 함수를 통해 해당 URL에 쿼리를 전송하면 이에 해당하는 데이터를 받으며, 이중 텍스트에 해당하는 내용만 불러오도록 합니다.

위의 과정을 거쳐 생성된 OTP를 제출하면, 우리가 원하는 데이터를 다운로드할 수 있습니다.

In [2]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_KS  = rq.post(down_url, {'code':otp}, headers=headers)
sector_KS = pd.read_csv(BytesIO(down_sector_KS.content), encoding='EUC-KR')

1. OTP를 제출할 URL을 down_url에 입력합니다.
2. `post()` 함수를 통해 위에서 부여받은 OTP 코드를 해당 URL에 제출합니다.
3. 받은 데이터의 content 부분을 `BytesIO()` 함수를 이용해 바이너리스트림 형태로 만든 후, `read_csv()` 함수를 통해 데이터를 읽어옵니다. 해당 데이터는 EUC-KR 형태로 인코딩 되어 있으므로 이를 선언해줍니다.

In [3]:
sector_KS

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4540,-155,-3.30,212573219300
1,006840,AK홀딩스,KOSPI,기타금융,25350,150,0.60,335825671350
2,027410,BGF,KOSPI,기타금융,4905,-25,-0.51,469490859855
3,282330,BGF리테일,KOSPI,유통업,141000,4500,3.30,2437030746000
4,138930,BNK금융지주,KOSPI,기타금융,5780,0,0.00,1883905721880
...,...,...,...,...,...,...,...,...
912,069260,휴켐스,KOSPI,화학,26000,-100,-0.38,1062843288000
913,000540,흥국화재,KOSPI,보험,3000,5,0.17,192727935000
914,000547,흥국화재2우B,KOSPI,보험,19950,-50,-0.25,3064320000
915,000545,흥국화재우,KOSPI,보험,7660,-240,-3.04,5882880000


위 과정을 통해 sector_KS 변수에는 산업별 현황 데이터가 저장되었습니다. 코스닥 시장의 데이터도 다운로드 받도록 하겠습니다.

In [4]:
gen_otp_data = {
  'mktId': 'KSQ', # 코스닥 입력
  'trdDd': '20210108',
  'money': '1',
  'csvxls_isNo': 'false',
  'name': 'fileDown',
  'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text         

down_sector_KQ  = rq.post(down_url, {'code':otp}, headers=headers)
sector_KQ = pd.read_csv(BytesIO(down_sector_KQ.content), encoding='EUC-KR')

In [5]:
sector_KQ

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2245,-45,-1.97,100581637195
1,054620,APS홀딩스,KOSDAQ,금융,7500,-150,-1.96,152956657500
2,265520,AP시스템,KOSDAQ,반도체,26000,-100,-0.38,376485902000
3,211270,AP위성,KOSDAQ,통신장비,8100,-250,-2.99,121619912400
4,035760,CJ ENM,KOSDAQ,방송서비스,156200,8500,5.75,3425333854800
...,...,...,...,...,...,...,...,...
1466,024060,흥구석유,KOSDAQ,유통,7510,-40,-0.53,112650000000
1467,010240,흥국,KOSDAQ,기계·장비,5910,0,0.00,72827133360
1468,189980,흥국에프엔비,KOSDAQ,음식료·담배,2010,-40,-1.95,77158125270
1469,037440,희림,KOSDAQ,기타서비스,3935,-15,-0.38,54784939125


코스피 데이터와 코스닥 데이터를 하나로 합치도록 합니다.

In [6]:
down_sector = pd.concat([sector_KS, sector_KQ]).reset_index(drop=True)
down_sector['종목명'] = down_sector['종목명'].str.strip()

종목명에 공백에 있는 경우가 있으므로 `strip()` 함수를 이용해 이를 제거해줍니다. 합쳐진 데이터를 csv 파일로 저장하겠습니다.

In [ ]:
import os
if not os.path.exists('data'):
    os.makedirs('data')

down_sector.to_csv('data/krx_sector.csv')

먼저 `if not()` 구문을 통해 data라는 이름의 폴더가 없으면 해당 이름으로 폴더를 생성해줍니다. 그 후 앞서 다운로드한 데이터를 data 폴더 안에 krx_sector.csv 이름으로 저장합니다. 해당 폴더를 확인해보면 데이터가 csv 형태로 저장되어 있습니다.

### 5.1.2 개별종목 지표 크롤링

개별종목 데이터를 크롤링하는 방법은 위와 매우 유사하며, 요청하는 쿼리 값에만 차이가 있습니다. 개발자 도구 화면을 열고 [CSV] 버튼을 클릭해 어떠한 쿼리를 요청하는지 확인합니다.

```{figure} image/05_crawl_practice_krx_ind.png
---
name: 05_crawl_practice_krx_ind
---
개별지표 OTP 생성 부분
```

이 중 tboxisuCd_finder_stkisu0_6, isu_Cd, isu_Cd2 등의 항목은 조회 구분의 개별추이 탭에 해당하는 부분이므로 우리가 원하는 전체 데이터를 받을 때는 필요하지 않은 요청값입니다. 이를 제외한 요청값을 산업별 현황 예제에 적용하면 해당 데이터 역시 손쉽게 다운로드할 수 있습니다.

In [7]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
  'searchType' : '1',
  'mktId' : 'ALL',
  'trdDd' : '20210108',
  'csvxls_isNo' : 'false',
  'name' : 'fileDown',
  'url' : 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}           
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text         

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_ind  = rq.post(down_url, {'code':otp}, headers=headers)

down_ind = pd.read_csv(BytesIO(down_ind.content), encoding='EUC-KR')
down_ind['종목명'] = down_ind['종목명'].str.strip()

In [8]:
down_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,BPS,PBR,주당배당금,배당수익률
0,060310,3S,2245,-45,-1.97,NaN,NaN,745.0,3.01,0,0.00
1,095570,AJ네트웍스,4540,-155,-3.30,982.0,4.62,6802.0,0.67,300,6.61
2,006840,AK홀딩스,25350,150,0.60,2168.0,11.69,62448.0,0.41,750,2.96
3,054620,APS홀딩스,7500,-150,-1.96,NaN,NaN,10530.0,0.71,0,0.00
4,265520,AP시스템,26000,-100,-0.38,671.0,38.75,7468.0,3.48,50,0.19


위 과정을 통해 down_ind 변수에는 개별종목 지표 데이터가 저장되었습니다. 해당 데이터 역시 csv 파일로 저장하겠습니다.

In [ ]:
down_ind.to_csv('data/krx_ind.csv')

```{note}
다운로드 받은 csv 파일을 확인해보면 아래 그림과 같이 인코딩 오류가 발생할 수 있습니다. 이는 파이썬의 기본 인코딩이 UTF-8인 반면, 윈도우의 인코딩은 EUC-KR 이기 때문에 발생하는 문제입니다.

해당 파일을 다시 파이썬으로 읽어오면 인코딩 문제가 발생하지 않습니다. 그러나 만일 윈도우에서 csv 파일 자체를 인코딩 오류 없이 사용하고 싶을 경우, `변수.to_csv('파일명.csv', encoding = 'EUC-KR')`를 통해 인코딩을 EUC-KR로 설정해주면 한글이 깨지지 않습니다.

```{figure} image/crawl_encoding_error.png
---
name: crawl_encoding_error
---
저장된 csv 파일의 인코딩 오류
```

### 5.1.3 최근 영업일 기준 데이터 받기

위 예제의 쿼리 항목 중 date와 schdate 부분을 원하는 일자로 입력하면(예: 20210104) 해당일의 데이터를 다운로드할 수 있으며, 최근 영업일 날짜를 입력하면 가장 최근의 데이터를 받을 수 있습니다. 그러나 매번 해당 항목을 입력하기는 번거로우므로 자동으로 반영되게 할 필요가 있습니다.

네이버 금융의 [국내증시 → 증시자금동향]에는 이전 2영업일에 해당하는 날짜가 있으며, 자동으로 날짜가 업데이트되어 편리합니다. 따라서 해당 부분을 크롤링해 쿼리 항목에 사용할 수 있습니다.

```
https://finance.naver.com/sise/sise_deposit.nhn
```

```{figure} image/05_crawl_practice_recentdate.png
---
name: 05_crawl_practice_recentdate
---
최근 영업일 부분
```

개발자도구 화면을 이용해 해당 데이터가 있는 부분을 확인해보면 [div 태그의 subtop_sise_graph2 클래스 → ul 태그의 subtop_chart_note 클래스  → li 태그 → span 태그의 tah 클래스]에 위치해 있다는 걸 알 수 있습니다. 이를 이용해 해당 데이터를 크롤링하도록 하겠습니다.

In [9]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2021.04.09


1. 페이지의 url을 저장합니다.
2. `get()` 함수를 통해 해당 페이지 내용을 받습니다.
3. `BeautifulSoup()` 함수를 이용해 해당 페이지의 HTML 내용을 읽어옵니다.
4. `select_one()` 함수를 통해 해당 태그의 데이터를 추출하며, text를 이용해 텍스트 데이터만을 추출합니다.

위 과정을 통해 **|  yyyy.mm.dd** 형식의 데이터가 선택됩니다. 이 중 숫자 부분만을 뽑아 **yyyymmdd** 형태로 만들어주도록 하겠습니다.

In [10]:
import re
biz_day = re.findall("[0-9]+", parse_day)
biz_day = "".join(biz_day)

print(biz_day)

20210409


1. `findall()` 함수 내에 정규표현식을 이용해 숫자에 해당하는 부분만을 추출합니다.**[0-9]+**는 모든 숫자를 의미하는 표현식입니다.
2. `join()` 함수를 통해 숫자를 합쳐줍니다.

이를 통해 우리가 원하는 yyyymmdd 형태의 날짜가 만들어졌습니다. 이를 쿼리의 trdDd에 입력하면 업종분류 현황과 개별종목 지표를 최근 영업일 기준으로 다운로드하게 됩니다. 전체 코드는 다음과 같습니다.

In [32]:
import requests as rq
from bs4 import BeautifulSoup
import re
from io import BytesIO
import pandas as pd
import os

if not os.path.exists('data'):
    os.makedirs('data')

# 최근 영업일 구하기
url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content, "html5lib")
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text
biz_day = re.findall("[0-9]+", parse_day)
biz_day = "".join(biz_day)

# 코스피 업종분류 데이터 다운로드
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'

gen_otp_data = {
  'mktId': 'STK',
  'trdDd': '20210108',
  'money': '1',
  'csvxls_isNo': 'false',
  'name': 'fileDown',
  'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}           

otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text   

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_KS  = rq.post(down_url, {'code':otp}, headers=headers)

sector_KS = pd.read_csv(BytesIO(down_sector_KS.content), encoding='EUC-KR')

# 코스닥 업종분류 데이터 다운로드
gen_otp_data = {
  'mktId': 'KSQ',
  'trdDd': biz_day,
  'money': '1',
  'csvxls_isNo': 'false',
  'name': 'fileDown',
  'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text         

down_sector_KQ  = rq.post(down_url, {'code':otp}, headers=headers)
sector_KQ = pd.read_csv(BytesIO(down_sector_KQ.content), encoding='EUC-KR')

# 합치기
down_sector = pd.concat([sector_KS, sector_KQ]).reset_index(drop=True)
down_sector['종목명'] = down_sector['종목명'].str.strip()

# 저장하기
down_sector.to_csv('data/krx_sector.csv')

# 개별종목 지표 OTP 발급
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'

gen_otp_data = {
  'searchType' : '1',
  'mktId' : 'ALL',
  'trdDd' : biz_day,
  'csvxls_isNo' : 'false',
  'name' : 'fileDown',
  'url' : 'dbms/MDC/STAT/standard/MDCSTAT03501'
}

otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text         

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_ind  = rq.post(down_url, {'code':otp}, headers=headers)

down_ind = pd.read_csv(BytesIO(down_ind.content), encoding='EUC-KR')
down_ind['종목명'] = down_ind['종목명'].str.strip()

# 저장하기
down_ind.to_csv('data/krx_ind.csv')

### 5.1.4 거래소 데이터 정리하기

위에서 다운로드한 데이터는 중복된 열이 있으며, 불필요한 데이터 역시 있습니다. 따라서 하나의 테이블로 합친 후 정리할 필요가 있습니다. 먼저 다운로드한 csv 파일을 읽어옵니다.

In [2]:
down_sector = pd.read_csv('data/krx_sector.csv', index_col = 0)
down_ind = pd.read_csv('data/krx_ind.csv', index_col = 0)

`read_csv()` 함수를 이용해 csv 파일을 불러옵니다. index_col = 0을 통해 첫 번째 열을 행 이름으로 지정합니다.

In [3]:
down_sector.columns & down_ind.columns

Index(['종목코드', '종목명', '종가', '대비', '등락률'], dtype='object')

먼저 두 데이터 간 중복되는 열 이름을 살펴보면 종목코드와 종목명 등이 확인됩니다.

In [4]:
print( set(down_sector['종목명']).symmetric_difference(set(down_ind['종목명'])) )

{'이스트아시아홀딩스', '케이탑리츠', '삼일제약', '이리츠코크렙', '신한알파리츠', 'SNK', '이지스레지던스리츠', '롯데리츠', '하이골드12호', '오가닉티코스메틱', '윙입푸드', '미래에셋맵스리츠', 'TS인베스트먼트', '크리스탈신소재', '글로벌에스엠', '코오롱티슈진', '한국ANKOR유전', '판타지오', 'GRT', '헝셩그룹', '바다로19호', 'TS인베스트먼트 [락]', '맥쿼리인프라', '제이알글로벌리츠', '엑세스바이오', '컬러레이', '로스웰', '프레스티지바이오파마', '코람코에너지리츠', 'NH프라임리츠', '엘브이엠씨홀딩스', '판타지오 (액)', '미투젠', '골든센츄리', 'SBI핀테크솔루션즈', '에이리츠', '삼일제약 (액)', '소마젠(Reg.S)', '한국패러랠', '잉글우드랩', '에스앤씨엔진그룹', '이지스밸류리츠', 'ESR켄달스퀘어리츠', '씨케이에이치', '베트남개발1', '모두투어리츠', 'JTC', '하이골드3호', '뉴프라이드', '맵스리얼티1', '네오이뮨텍(Reg.S)'}


두 데이터의 종목명 열을 세트 형태로 변경한 후 `symmetric_difference()` 함수를 통해 공통으로 없는 종목명, 즉 하나의 데이터에만 있는 종목을 살펴보면 위와 같습니다. 해당 종목들은 선박펀드, 광물펀드, 해외종목 등 일반적이지 않은 종목들이므로 제외하는 것이 좋습니다. 따라서 둘 사이에 공통적으로 존재하는 종목을 기준으로 데이터를 합쳐주겠습니다

In [5]:
KOR_ticker = pd.merge(down_sector, down_ind,
                      on = (down_sector.columns & down_ind.columns).tolist(), how = 'inner')

`merge()` 함수는 on을 기준으로 두 데이터를 하나로 합치며, 공통으로 존재하는 **종목코드, 종목명, 종가, 대비, 등락률**을 기준으로 입력해줍니다. 또한 둘 사이에 공통적으로 존재하는 종목을 기준으로 데이터를 합쳐주어야 하므로, 방법(how)에 inner를 입력해 줍니다.

In [6]:
KOR_ticker = KOR_ticker.sort_values(by = ['시가총액'], ascending = False)

KOR_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,EPS,PER,BPS,PBR,주당배당금,배당수익률
404,005930,삼성전자,KOSPI,전기전자,83200,-400,-0.48,496685908160000,3166.0,26.28,37528.0,2.22,1416,1.70
134,000660,SK하이닉스,KOSPI,전기전자,137500,-2500,-1.79,100100325187500,2943.0,46.72,65836.0,2.09,1000,0.73
94,035420,NAVER,KOSPI,서비스업,385500,2000,0.52,63323538772500,4006.0,96.23,35223.0,10.94,376,0.10
405,005935,삼성전자우,KOSPI,전기전자,75100,100,0.13,61798791170000,NaN,NaN,NaN,NaN,1417,1.89
86,051910,LG화학,KOSPI,화학,817000,5000,0.62,57673944231000,4085.0,200.00,217230.0,3.76,2000,0.24


데이터를 시가총액 기준으로 내림차순 정렬할 필요도 있습니다. `sort_values()` 함수를 통해 데이터를 정렬해주며, 기준으로 시가총액을 선택합니다. 파이썬은 기본적으로 오름차순으로 정렬을 하므로 ascending = False, 즉 내림차순으로 데이터를 정렬합니다. 결과적으로 시가총액 기준 내림차순으로 해당 데이터가 정렬됩니다.

마지막으로 스팩, 우선주 종목 역시 제외해야 합니다.

In [7]:
print( KOR_ticker[KOR_ticker['종목명'].str.contains('스팩')]['종목명'].values )

['유안타제6호스팩' '삼성스팩2호' '케이비제18호스팩' '미래에셋대우스팩4호' '엔에이치스팩17호' '하나금융17호스팩'
 '미래에셋대우스팩3호' '케이비제20호스팩' '유안타제8호스팩' '엔에이치스팩16호' 'DB금융스팩8호' '유안타제5호스팩'
 '한화에스비아이스팩' 'SK6호스팩' '대신밸런스제8호스팩' '유안타제7호스팩' '케이비17호스팩' '교보10호스팩'
 'IBKS제13호스팩' '대신밸런스제7호스팩' '미래에셋대우스팩 5호' 'SK4호스팩' '신한제6호스팩' '하이제5호스팩'
 '에이치엠씨제5호스팩' '상상인이안1호스팩' '하나머스트제6호스팩' 'DB금융스팩9호' '하나금융15호스팩' '한국9호스팩'
 '유안타제4호스팩' '한화플러스제1호스팩' '엔에이치스팩18호' '신한제7호스팩' '하나금융14호스팩' 'IBKS제14호스팩'
 '에이치엠씨제4호스팩' '신영스팩6호' 'SK5호스팩' '하나머스트7호스팩' '케이비제19호스팩' '대신밸런스제9호스팩'
 '유진스팩5호' '하나금융16호스팩' '교보9호스팩' '상상인이안제2호스팩' '키움제5호스팩' '이베스트스팩5호'
 'IBKS제15호스팩' '유진스팩4호' '신영스팩5호' '한국제8호스팩' '엔에이치스팩13호' 'IBKS제12호스팩'
 '이베스트이안스팩1호' '유진스팩6호']


In [9]:
print( KOR_ticker[KOR_ticker['종목명'].str.contains('[0-9]+호')]['종목명'].values )

['유안타제6호스팩' '삼성스팩2호' '케이비제18호스팩' '미래에셋대우스팩4호' '엔에이치스팩17호' '하나금융17호스팩'
 '미래에셋대우스팩3호' '케이비제20호스팩' '유안타제8호스팩' '엔에이치스팩16호' 'DB금융스팩8호' '유안타제5호스팩'
 'SK6호스팩' '대신밸런스제8호스팩' '유안타제7호스팩' '케이비17호스팩' '교보10호스팩' 'IBKS제13호스팩'
 '대신밸런스제7호스팩' '미래에셋대우스팩 5호' 'SK4호스팩' '신한제6호스팩' '하이제5호스팩' '에이치엠씨제5호스팩'
 '상상인이안1호스팩' '하나머스트제6호스팩' 'DB금융스팩9호' '하나금융15호스팩' '한국9호스팩' '유안타제4호스팩'
 '한화플러스제1호스팩' '엔에이치스팩18호' '신한제7호스팩' '하나금융14호스팩' 'IBKS제14호스팩' '에이치엠씨제4호스팩'
 '신영스팩6호' 'SK5호스팩' '하나머스트7호스팩' '케이비제19호스팩' '대신밸런스제9호스팩' '유진스팩5호'
 '하나금융16호스팩' '교보9호스팩' '상상인이안제2호스팩' '키움제5호스팩' '이베스트스팩5호' 'IBKS제15호스팩'
 '유진스팩4호' '신영스팩5호' '한국제8호스팩' '엔에이치스팩13호' 'IBKS제12호스팩' '이베스트이안스팩1호'
 '케이프이에스제4호' '유진스팩6호']


In [8]:
print( KOR_ticker[KOR_ticker['종목코드'].str[-1:] != '0']['종목명'].values )

['삼성전자우' '현대차2우B' 'LG화학우' '현대차우' 'LG생활건강우' 'LG전자우' '아모레퍼시픽우' '삼성SDI우'
 '미래에셋증권2우B' '한국금융지주우' '삼성화재우' '신영증권우' '대신증권우' '삼성전기우' '한화3우B' '금호석유우'
 'CJ4우(전환)' '아모레G3우(전환)' 'CJ제일제당 우' 'SK이노베이션우' '현대차3우B' 'LG우' '두산퓨얼셀1우'
 '삼성물산우B' '신풍제약우' 'NH투자증권우' 'S-Oil우' 'SK케미칼우' 'DL이앤씨우' '두산우' 'SK우' '아모레G우'
 '한화투자증권우' '대신증권2우B' 'CJ우' '솔루스첨단소재1우' '코오롱인더우' '미래에셋증권우' '녹십자홀딩스2우'
 '유한양행우' 'DL우' '한화솔루션우' '호텔신라우' '부국증권우' '유안타증권우' '롯데지주우' 'GS우' '덕성우'
 'SK디스커버리우' '두산퓨얼셀2우B' '대교우B' '롯데칠성우' '두산2우B' '유화증권우' '대한항공우' '솔루스첨단소재2우B'
 'LG하우시스우' '삼성중공우' 'BYC우' '티와이홀딩스우' '코오롱우' '남양유업우' '세방우' '대상우' '하이트진로2우B'
 '대덕전자1우' 'SK증권우' '대한제당우' '한진칼우' '태영건설우' '한화우' '넥센타이어1우B' '삼양홀딩스우' '코리아써우'
 '현대건설우' '삼양사우' '일양약품우' 'SK네트웍스우' 'NPC우' '한양증권우' 'DB하이텍1우' '깨끗한나라우'
 '유유제약1우' '크라운제과우' '넥센우' '서울식품우' '성신양회우' '대덕1우' '대상홀딩스우' '남선알미우' '태양금속우'
 '크라운해태홀딩스우' '계양전기우' '동원시스템즈우' '금호산업우' '코오롱글로벌우' '현대비앤지스틸우' '금강공업우'
 '루트로닉3우C' 'CJ씨푸드1우' '하이트진로홀딩스우' '노루페인트우' '흥국화재우' '대원전선우' 'JW중외제약우'
 'KG동부제철우' '대호특수강우' 'JW중외제약2우B' '동부건설우' '진흥기업우B' '코리아써키트2우B' '성문전자우'
 '유유

1. `contains()` 함수를 통해 종목명에 '스팩'이 들어가는 종목을 찾을 수 있습니다
2. 간혹 스팩이지만 '스팩'이라는 단어가 들어가지 않은 종목이 있습니다. 이는 정규표현식을 통해 'n호'라는 형태의 종목명으로 찾을 수 있습니다.
3. 종목코드 끝이 0이 아닌 종목을 통해 우선주 종목을 찾을 수 있습니다.

해당 종목을 제외한 데이터만 선택해주도록 하겠습니다.

In [10]:
KOR_ticker = KOR_ticker[~KOR_ticker['종목명'].str.contains('스팩')]
KOR_ticker = KOR_ticker[~KOR_ticker['종목명'].str.contains('[0-9]+호')]
KOR_ticker = KOR_ticker[KOR_ticker['종목코드'].str[-1:] == '0']

1. 종목명에 '스팩'이 포함되지 않은 종목을 선택합니다.
2. 종목코드 끝이 0인 종목, 즉 보통주만을 선택합니다.

마지막으로 행 이름을 초기화한 후 정리된 데이터를 csv 파일로 저장합니다.

In [11]:
KOR_ticker = KOR_ticker.reset_index(drop=True)
KOR_ticker.to_csv('data/KOR_ticker.csv')

## 5.2 WICS 기준 섹터정보 크롤링

일반적으로 주식의 섹터를 나누는 기준은 MSCI와 S&P가 개발한 GICS를 가장 많이 사용합니다. 국내 종목의 GICS 기준 정보 역시 한국거래소에서 제공하고 있으나, 이는 독점적 지적재산으로 명시했기에 사용하는 데 무리가 있습니다. 그러나 지수제공업체인 와이즈인덱스에서는 GICS와 비슷한 WICS 산업분류를 발표하고 있습니다. WICS를 크롤링해 필요한 정보를 수집해보겠습니다.

```
http://www.wiseindex.com/Index
```

먼저 웹페이지에 접속해 왼쪽에서 [WISE SECTOR INDEX → WICS → 에너지]를 클릭합니다. 그 후 [Components] 탭을 클릭하면 해당 섹터의 구성종목을 확인할 수 있습니다.

```{figure} image/05_wise.png
---
name: 05_wise
---
WICS 기준 구성종목
```

개발자도구 화면({numref}`05_wics2`)을 통해 해당 페이지의 데이터전송 과정을 살펴보도록 하겠습니다.

```{figure} image/05_wics2.png
---
name: 05_wics2
---
WICS 페이지 개발자도구 화면
```

일자를 선택하면 [Network] 탭의 GetIndexComponets 항목을 통해 데이터 전송 과정이 나타납니다. Request URL은 데이터를 가져오는 주소이며, 이를 분석하면 다음과 같습니다.

```
http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20210210&sec_cd=G10
```

1. http://www.wiseindex.com/Index/GetIndexComponets: 데이터를 요청하는 url 입니다.
2. ceil_yn = 0: 실링 여부를 나타내며, 0은 비실링을 의미합니다.
3. dt=20210210: 조회일자를 나타냅니다.
4. sec_cd=G10: 섹터 코드를 나타냅니다.

이번엔 위 주소의 페이지를 열어보겠습니다.

```{figure} image/05_wics3.png
---
name: 05_wics3
---
WICS 데이터 페이지
```

글자들은 페이지에 출력된 내용이지만 매우 특이한 형태로 구성되어 있는데 이것은 JSON 형식의 데이터입니다. 기존에 우리가 살펴보았던 대부분의 웹페이지는 HTML 형식으로 표현되어 있습니다. HTML 형식은 문법이 복잡하고 표현 규칙이 엄격해 데이터의 용량이 커지는 단점이 있습니다. 반면 JSON 형식은 문법이 단순하고 데이터의 용량이 작아 빠른 속도로 데이터를 교환할 수 있습니다. 파이썬에서는 json 패키지를 사용해 매우 손쉽게 JSON 형식의 데이터를 크롤링할 수 있습니다.

In [20]:
import json
import requests as rq

url = 'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20210210&sec_cd=G10'
data = rq.get(url).json()

In [21]:
print(data.keys())

dict_keys(['info', 'list', 'sector', 'size'])


`get()` 함수를 통해 해당 페이지의 내용을 받아온 후, `json()` 함수를 통해 JSON 형태의 데이터를 읽어옵니다.

list 항목에는 해당 섹터의 구성종목 정보가 있으며, sector 항목을 통해 다른 섹터의 코드도 확인할 수 있습니다. list 부분의 데이터를 데이터프레임 형태로 변경하도록 하겠습니다.

In [22]:
data_pd = pd.json_normalize(data['list'])
data_pd.head()

,IDX_CD,IDX_NM_KOR,ALL_MKT_VAL,CMP_CD,CMP_KOR,MKT_VAL,WGT,S_WGT,CAL_WGT,SEC_CD,SEC_NM_KOR,SEQ,TOP60,APT_SHR_CNT
0,G10,WICS 에너지,25711965,096770,SK이노베이션,15352982,59.71,59.71,1.0,G10,에너지,1,2,51780716
1,G10,WICS 에너지,25711965,010950,S-Oil,3444921,13.40,73.11,1.0,G10,에너지,2,2,41655633
2,G10,WICS 에너지,25711965,267250,현대중공업지주,2353362,9.15,82.26,1.0,G10,에너지,3,2,8847226
3,G10,WICS 에너지,25711965,078930,GS,1910712,7.43,89.69,1.0,G10,에너지,4,2,49245150
4,G10,WICS 에너지,25711965,006120,SK디스커버리,701545,2.73,92.42,1.0,G10,에너지,5,2,10470820


pandas 패키지의 `json_normalize()` 함수를 이용하면 JSON 형태의 데이터를 데이터프레임 형태로 매우 쉽게 변경할 수 있습니다. 

`for loop` 구문을 이용해 URL의 sec_cd=에 해당하는 부분만 변경하면 모든 섹터의 구성종목을 매우 쉽게 얻을 수 있습니다.

In [24]:
import time
import json
import requests as rq
import pandas as pd
from tqdm import tqdm

sector_code = ['G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45']

data_sector = {}

for i in tqdm(sector_code):
  url = 'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt='+biz_day+'&sec_cd='+i
  data = rq.get(url).json()
  data_pd = pd.json_normalize(data['list'])

  data_sector[i] = data_pd

  time.sleep(2)

sector_list = [v for k,v in data_sector.items()] 
KOR_sector = pd.concat(sector_list).reset_index(drop=True)

KOR_sector.to_csv('data/KOR_sector.csv')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:26<00:00,  2.62s/it]


1. for loop 구문을 이용해 모든 섹터의 구성종목을 다운로드 받으며, tqdm() 함수를 통해 진행상황을 출력합니다.
2. 딕셔너리 중 values에 해당하는 부분만 선택해 sector_list에 저장합니다. (k는 key, v는 value를 의미합니다.)
3. concat() 함수를 이용해 데이터를 합쳐준 후, index를 초기화 합니다.
4. 해당 데이터를 csv 파일로 저장해주도록 합니다.

## 5.3 CMD 파일을 이용한 자동화

매번 데이터를 다운로드 받을 때 마다 파이썬을 연 후 코드를 실행하는 것은 번거롭습니다. 윈도우 내에서는 명령어 스크립트(cmd) 파일을 이용해 이러한 작업을 클릭만으로 해결할 수 있습니다.

먼저 위에서 살펴본 최근 영업일 및 WICS 기준 섹터정보 크롤링을 하나의 코드로 작성한 후 파이썬 스크립트 파일인 **down_data.py** 로 저장합니다.

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import json
from tqdm import tqdm
import re

if not os.path.exists('data'):
    os.makedirs('data')

# 최근 영업일 구하기
url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content, "html5lib")
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text
biz_day = re.findall("[0-9]+", parse_day)
biz_day = "".join(biz_day)

print('최근 영업일:' + biz_day)

# 섹터정보 다운로드
print('섹터 정보를 다운로드 합니다.')

sector_code = ['G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45']

data_sector = {}

for i in tqdm(sector_code):
  url = 'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt='+biz_day+'&sec_cd='+i
  data = rq.get(url).json()
  data_pd = pd.json_normalize(data['list'])

  data_sector[i] = data_pd

  time.sleep(2)

sector_list = [v for k,v in data_sector.items()] 
KOR_sector = pd.concat(sector_list).reset_index(drop=True)

KOR_sector.to_csv('data/KOR_sector.csv')
input("다운로드를 완료하였습니다. 엔터를 누르면 프로그램이 종료됩니다.")

먼저 해당 스크립트를 실행해 코드가 정상적으로 작동되는지 확인해 봅니다.

윈도우의 명령 프롬프트에서 `pyhon 스크립트.py`를 입력하면 해당 파이썬 스크립트가 실행됩니다. 이러한 작업을 클릭만으로 해결하기 위해, 메모장에 `python down_data.py` 를 입력한 후 **down_data_cmd.cmd** 로 저장합니다.

```{figure} image/crawl_cmd.png
---
name: crawl_cmd
---
명령어 스크립트 작성
```

폴더에는 파이썬 스크립트와 명령어 스크립트 파일이 저장되어 있습니다. 명렁어 스크립트를 실행해 보면 위에서 작성한 파이썬 스크립트가 실행되어, 최근 영업일 기준 섹터 정보가 다운로드 됩니다.

```{figure} image/crawl_cmd2.png
---
name: crawl_cmd2
---
파이썬 스크립트 및 명령어 스크립트 파일
```

```{figure} image/crawl_cmd3.png
---
name: crawl_cmd3
---
명령어 스크립트 실행
```
이 외에도 위에서 살펴본 한국거래소 데이터 다운로드 및 정리, 다음 장에서 살펴볼 주가와 재무제표 다운로드 및 정리 등 모든 작업을 파이썬 스크립트 저장 및 이를 실행하는 명령어 스크립트 작성을 통해, 파이썬을 열지 않고도 클릭만으로 해결할 수 있습니다.